In [63]:
# pip install openpyxl

In [64]:
import pandas as pd
# import openpyxl

In [65]:
sumula = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Sumula_Novo_Padrao.xlsx",
    sheet_name=None,
    header=1,
    engine="calamine"
)

In [66]:
# sumula["Individual"] = sumula["Individual"][sumula["Individual"]["Boleiro"].notna()]
sumula["Individual"] = sumula["Individual"].query("Boleiro == Boleiro") # Mesmo efeito do notna()

sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                      "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]] = \
    sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                          "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]].fillna(0).astype("int")
# sumula["Individual"]

In [67]:
print(sumula.keys())

dict_keys(['Baba', 'Boleiros', 'Individual', 'Times', 'Pizza', 'Premios', 'DefesasPenaltis', 'Ouros', 'Contra', 'SelecaodoMes', 'Cartoes'])


In [68]:
lista_resultados_pizza = []
sumula["Pizza"]["Pizza"] = 1
for col in sumula["Pizza"].columns:
    if col.startswith("Colete"):
        resultados_pizza = sumula["Individual"].merge(sumula["Pizza"], how="left", left_on=["Baba", "Colete"], right_on=["Baba", col])
        lista_resultados_pizza.append(resultados_pizza)
sumula_pizza = pd.concat(lista_resultados_pizza, ignore_index=True)
sumula_pizza["Pizza"] = sumula_pizza["Pizza"].fillna(0).astype(int)

sumula_pizza = sumula_pizza.drop(
    columns=[col for col in sumula_pizza.columns if col.startswith("Colete") and col != "Colete"]
)

# Quando há mais de um "colete" vencedor, os resultados são duplicados, sendo necessário remover as duplicatas
sumula_pizza = sumula_pizza.sort_values(by="Pizza", ascending=False).drop_duplicates(subset=["Baba", "Boleiro"], keep="first")

sumula_pizza = sumula_pizza.sort_values(by=["Baba", "Colete", "Boleiro"], ascending=True).reset_index(drop=True)

In [69]:
sumula_defesaspenalti = sumula_pizza.merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
sumula_defesaspenalti["Defesas de Penalti"] = sumula_defesaspenalti["Defesas de Penalti"].fillna(0).astype("int")
# sumula_defesaspenalti

In [70]:
# Ouros (O), Gols Contra (C), Cartões (C)
sumula_occ = (sumula_defesaspenalti
              .merge(sumula["Ouros"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Contra"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Cartoes"], how="left", on=["Baba", "Boleiro"])
              )
sumula_occ[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]] = \
sumula_occ[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]].fillna(0).astype("int")
# sumula_occ

In [71]:
df_premios = sumula["Premios"].melt(id_vars=["Baba", "Premio"], value_name="Boleiro")[["Baba", "Premio", "Boleiro"]]
df_premios = df_premios.dropna(subset=["Boleiro"]).reset_index(drop=True)
df_premios["Valor"] = 1
df_premios = df_premios.pivot_table(index=["Baba", "Boleiro"], columns="Premio", values="Valor", fill_value=0)

df_premios.columns = [f"Premio {col}" if col in ["Gol", "Drible", "Defesa",
                                                 "Inacreditavel"] else col for col in df_premios.columns]
df_premios = df_premios.reset_index()

sumula_premios = sumula_occ.merge(df_premios, how="left", left_on=["Baba", "Boleiro"], right_on=["Baba", "Boleiro"])
sumula_premios = sumula_premios.fillna(0).astype({"Premio Gol":int, "Premio Drible":int, "Premio Defesa":int,
                                          "Premio Inacreditavel":int, "Bola Cheia":int, "Bola Murcha":int,
                                          "Clean Sheets":int, "Roleta":int})
# sumula_premios

In [72]:
sumula_presenca = sumula_premios.copy()
sumula_presenca["Presenca"] = 1
# sumula_presenca

In [73]:
sumula_ranking = sumula_presenca.copy()
sumula_ranking["Ranking"] = (
    sumula_ranking["Gols"]
    + sumula_ranking["Assistencias"]
    + sumula_ranking["Vitorias"] * 3
    + sumula_ranking["Empates"] * 3
    + sumula_ranking["Mensalidade"]
    + sumula_ranking["Colaboracao"]
    + sumula_ranking["Defesas de Penalti"]
    + sumula_ranking["Bola Cheia"] * 3
    + sumula_ranking["Clean Sheets"] * 3
    + sumula_ranking["Premio Defesa"] * 3
    + sumula_ranking["Premio Drible"] * 3
    + sumula_ranking["Premio Gol"] * 3
)

sumula_ranking["Ranking sem Colaboracao"] = sumula_ranking["Ranking"] - sumula_ranking["Colaboracao"]
# sumula_ranking

In [74]:
df_boleiros = sumula["Boleiros"].copy()
df_baba = sumula["Baba"][["Baba", "Data"]].copy()

df_baba["Mes"] = df_baba["Data"].dt.month_name(locale="Portuguese_Brazil")
df_baba["Ano"] = df_baba["Data"].dt.year

df_validacao_ultimo_baba_mes = df_baba.copy()
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)

df_validacao_ultimo_baba_mes["Total Babas"] = (
    df_validacao_ultimo_baba_mes.groupby(["Ano", "Mes"])["Baba"].transform("count")
)

df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba", "Total Babas"]
]
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes[["Baba", "Total Babas"]].reset_index(drop=True)


df_ultimo_baba_mes = sumula_ranking.merge(df_baba, on=["Baba"], how="left")
df_ultimo_baba_mes = df_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)
df_ultimo_baba_mes = df_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba"]
]

df_ultimo_baba_mes = df_ultimo_baba_mes.rename(columns={"Baba":"Ultimo Baba"})

df_ranking_sem_colab = sumula_ranking[["Baba", "Boleiro", "Ranking sem Colaboracao", "Vitorias", "Gols", "Assistencias"]].copy()
df_pontuacao_do_mes = df_ranking_sem_colab \
                    .merge(df_boleiros, on="Boleiro", how="left") \
                    .merge(df_baba, on=["Baba"], how="left")                    
df_pontuacao_do_mes["Mes"] = df_pontuacao_do_mes["Data"].dt.month_name(locale="Portuguese_Brazil")
df_pontuacao_do_mes["Ano"] = df_pontuacao_do_mes["Data"].dt.year
df_pontuacao_do_mes = df_pontuacao_do_mes.groupby(["Ano", "Mes", "Boleiro", "Posicao"])[
    ["Ranking sem Colaboracao", "Vitorias", "Gols", "Assistencias"]
].sum().reset_index()

df_pontuacao_do_mes = df_pontuacao_do_mes.merge(df_ultimo_baba_mes, on=["Ano", "Mes"], how="left")

df_pontuacao_do_mes = df_pontuacao_do_mes.sort_values(["Ranking sem Colaboracao", "Vitorias", "Gols", "Assistencias"], ascending=False)
df_pontuacao_do_mes = df_pontuacao_do_mes[
    df_pontuacao_do_mes["Ultimo Baba"].isin(df_validacao_ultimo_baba_mes["Baba"])
]

df_pontuacao_do_mes = df_pontuacao_do_mes \
    .merge(df_validacao_ultimo_baba_mes, left_on="Ultimo Baba", right_on="Baba", how="left") \
    .drop(columns="Baba")

# df_pontuacao_do_mes

In [75]:
ordem_posicoes = ["Goleiro", "Zagueiro", "Ala", "Meia", "Atacante"]
ordem_meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

df_pontuacao_do_mes["Posicao"] = pd.Categorical(df_pontuacao_do_mes["Posicao"], categories=ordem_posicoes, ordered=True)
df_pontuacao_do_mes["Mes"] = pd.Categorical(df_pontuacao_do_mes["Mes"], categories=ordem_meses, ordered=True)

df_selecao_do_mes = (
    df_pontuacao_do_mes[df_pontuacao_do_mes["Total Babas"] >= 3]
    .sort_values(
        by=["Ano", "Mes", "Posicao", "Ranking sem Colaboracao", "Vitorias", "Gols", "Assistencias", "Boleiro"],
        ascending=[True, False, True, False, False, False, False, True]
    )
    .groupby(["Ano", "Mes", "Ultimo Baba", "Posicao"], as_index=False, observed=True)
    .head(1)
    .reset_index(drop=True)
)

remover_colunas = ["Vitorias", "Gols", "Assistencias"]
ordenar_selecao_do_mes = ["Ano", "Mes", "Ultimo Baba", "Boleiro", "Posicao", "Pontuacao"]

df_selecao_do_mes = df_selecao_do_mes.rename(columns={"Ranking sem Colaboracao":"Pontuacao"}).drop(columns=remover_colunas)
df_selecao_do_mes = df_selecao_do_mes[ordenar_selecao_do_mes]

df_selecao_do_mes

,Ano,Mes,Ultimo Baba,Boleiro,Posicao,Pontuacao
0,2025,Janeiro,T10B04,Matheus,Goleiro,87
1,2025,Janeiro,T10B04,Leandro,Zagueiro,74
2,2025,Janeiro,T10B04,Carlos,Ala,85
3,2025,Janeiro,T10B04,Júlio,Meia,66
4,2025,Janeiro,T10B04,Zé,Atacante,77


In [76]:
sumula_ranking.columns

Index(['Baba', 'Boleiro', 'Colete', 'Gols', 'Assistencias', 'Vitorias',
       'Derrotas', 'Empates', 'Pontos no Baba', 'Mensalidade', 'Colaboracao',
       'Pontos de Copa', 'Pizza', 'Defesas de Penalti', 'Ouros', 'Gols Contra',
       'Cartao Amarelo', 'Cartao Azul', 'Cartao Vermelho', 'Bola Cheia',
       'Bola Murcha', 'Clean Sheets', 'Premio Defesa', 'Premio Drible',
       'Premio Gol', 'Premio Inacreditavel', 'Roleta', 'Presenca', 'Ranking',
       'Ranking sem Colaboracao'],
      dtype='object')

In [87]:
df_ultima_pizza = sumula_ranking[["Baba", "Boleiro", "Pizza"]].sort_values("Baba", ascending=False).copy()
lista_ultima_pizza = df_ultima_pizza[df_ultima_pizza["Pizza"] == 1].groupby("Boleiro")["Baba"].first()
df_ultima_pizza["Ultima Pizza"] = df_ultima_pizza["Boleiro"].map(lista_ultima_pizza).fillna("Não ganhou")
df_ultima_pizza = df_ultima_pizza[["Boleiro", "Ultima Pizza"]].drop_duplicates().reset_index(drop=True)
df_ultima_pizza


,Boleiro,Ultima Pizza
0,Zé,T10B04
1,Matheus,T10B04
2,Anderson,T10B04
3,Carlos,T10B04
4,Henrique,T10B04
5,Lúcio,T10B04
6,André,T10B04
7,Júlio,T10B04
8,Leandro,T10B04
9,Pedro,T10B04


In [78]:
# duplas_final = pd.DataFrame()
# for coluna_boleiro in time.columns:
#     if "Boleiro" in coluna_boleiro:
#         duplas = (sumula[["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"]]
#                 .merge(time, how="left", left_on=["Baba", "Colete", "Boleiro"], right_on=["Baba", "Colete", coluna_boleiro])
#                 .melt(id_vars=["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"],
#             value_vars=["Boleiro1", "Boleiro2", "Boleiro3", "Boleiro4", "Boleiro5", "Boleiro6"],
#             var_name="Origem",
#             value_name="SegundoBoleiro")
#             .drop(columns="Origem"))
#         duplas = duplas[duplas["Boleiro"] != duplas["SegundoBoleiro"]].reset_index(drop=True)
#         duplas_final = pd.concat([duplas_final, duplas], ignore_index=True)
# duplas_final = duplas_final.rename(columns={"SegundoBoleiro":"Boleiro2"})

In [79]:
# grupo_duplas = duplas_final.groupby(["Boleiro", "Boleiro2"]).agg(
#     Total_Pizza=("Pizza", "sum"),
#     Total_Vitorias=("Vitorias", "sum"),
#     Total_Derrotas=("Derrotas", "sum"),
#     Total_Empates=("Empates", "sum"),
#     Total_Babas=("Baba", "count")
# ).reset_index()
# grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
# grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
# grupo_duplas["M_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Babas"]
# grupo_duplas = grupo_duplas[
#             [
#                 "Boleiro",
#                 "Boleiro2",
#                 "Total_Pizza",
#                 "Total_Vitorias",
#                 "Total_Derrotas",
#                 "Total_Empates",
#                 "Total_Partidas",
#                 "Total_Babas",
#                 "P_Vitorias",
#                 "P_Derrotas",
#                 "P_Empates",
#                 "P_VD",
#                 "M_Pizza",
#                 "M_Vitorias",
#                 "M_Derrotas",
#                 "M_Empates"
#             ]
#         ]

In [80]:
# grupo_duplas = grupo_duplas.sort_values(["Total_Babas", "Total_Partidas", "Total_Vitorias"], ascending=False).reset_index(drop=True)
# grupo_duplas.index = grupo_duplas.index + 1
# grupo_duplas.to_excel("grupo_duplas.xlsx", index=False)
# grupo_duplas

In [81]:
# ranking_completo = sumula.groupby("Boleiro").agg(
#     Artilharia=("Gols", "sum"),
#     Assistencias=("Assistencias", "sum"),
#     Babas_Conquistados=("Vitorias", "sum"),
#     Babas_Perdidos=("Derrotas", "sum"),
#     Empates=("Empates", "sum"),
#     Pizza=("Pizza", "sum"),
#     Presenca_no_Baba=("Baba", "count"),
#     PB=("Pontos no Baba", "sum"),
#     Mensalidade=("Mensalidade", "sum"),
#     Pontos_de_Colaboracao=("Colaboracao","sum"),
#     Copa=("Pontos de Copa","sum"),
#     Ouros=("Ouros","sum"),
#     Gols_Contra=("Gols Contra","sum"),
#     ).reset_index()
# # ranking_completo